In [46]:
import overpass
from IPython.display import display
import geopandas as gpd

def get_point_data(node_name: str, area_name: str, api: overpass.API, date: str = None):
    query = ""
    # if date is not None: # to jeszcze nie dziala :(
    #     query += f'[date:"{date}"];'
    # else:
    #     query += ''
    query += f'''
    area["name"="{area_name}"]->.a;
    (
        node["{node_name}"](area.a);
    );
    out center;
    '''
    resp = api.get(query)
    nodes = []
    for node in resp['features']:
        x, y = node["geometry"]["coordinates"]
        node_geom = gpd.points_from_xy([x], [y])[0]
        name = node["properties"][node_name]
        nodes.append({"geometry": node_geom, "type": name})
    return nodes

api = overpass.API()
node_names = ['amenity', 'building', 'highway', 'public_transport', 'government', 'leisure', 'office', 'natural']
area_name = 'Virginia Beach'
date = '2019-01-01T00:00:00Z'
amenities = get_point_data(node_names[0], area_name, api)
buildings = get_point_data(node_names[1], area_name, api)
highways = get_point_data(node_names[2], area_name, api)
public_transport = get_point_data(node_names[3], area_name, api)
government = get_point_data(node_names[4], area_name, api)
leisure = get_point_data(node_names[5], area_name, api)
office = get_point_data(node_names[6], area_name, api)
natural = get_point_data(node_names[7], area_name, api)

In [47]:

amenities.extend(highways)
amenities.extend(buildings)
amenities.extend(leisure)
amenities.extend(office)
amenities.extend(government)
amenities.extend(public_transport)
# amenities.extend(natural)
features_gdf = gpd.GeoDataFrame(data=[{'name': amen['type']} for amen in amenities], geometry=[amen['geometry'] for amen in amenities], crs='EPSG:4326')
print(features_gdf.head())
# print number of unique names
print(len(features_gdf['name'].unique()))

               name                    geometry
0  place_of_worship  POINT (-76.04799 36.70960)
1        grave_yard  POINT (-76.01854 36.59043)
2  place_of_worship  POINT (-76.08286 36.58601)
3  place_of_worship  POINT (-76.07354 36.55765)
4            school  POINT (-76.02160 36.59710)
123


In [48]:

# import data from VBOHCAR.xlsx to a pandas dataframe
import io
import pandas as pd
import requests
from os import listdir

# check for the vbohcar.xlsx file in the current directory
if 'VBOHCAR.xlsx' in listdir():
        # read the third sheet of the excel file
    df = pd.read_excel('VBOHCAR.xlsx', sheet_name=3)
else:
    # clone the excel file from github
    url = 'https://github.com/INFORMSJoC/2020.1022/blob/master/results/VBOHCAR.xlsx?raw=true'
    file = requests.get(url)
    file_bytes = io.BytesIO(file.content)
    # read the third sheet of the excel file
    df = pd.read_excel(file_bytes, sheet_name=3)

# convert the pandas dataframe to a geopandas dataframe and add a geometry column with crs set
ohca_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitude'], df['Latitude']), crs='EPSG:4326')

# delete unused columns
columns_to_drop = ['Incident_Location', 'MinimumResponseTime', 'ReceivedTime', 'ID_OHCA', 'X_OHCA', 'Y_OHCA', 'Z_OHCA']
for column in columns_to_drop:
    if column in ohca_gdf.columns:
        del ohca_gdf[column]

print(ohca_gdf.head())

    Latitude  Longitude                    geometry
0  36.862471 -76.024169  POINT (-76.02417 36.86247)
1  36.766897 -76.042337  POINT (-76.04234 36.76690)
2  36.766897 -76.042337  POINT (-76.04234 36.76690)
3  36.905880 -76.118769  POINT (-76.11877 36.90588)
4  36.620850 -76.090090  POINT (-76.09009 36.62085)


In [49]:

import h3

# create a dict hexagon_amenities = {hexagon_id: {amenity_name: count}}
hexagon_amenities = {}
for amen_dict in features_gdf.iterrows():
    amenity = amen_dict[1]['name']
    hexagon = h3.geo_to_h3(amen_dict[1]['geometry'].y, amen_dict[1]['geometry'].x, 9)
    if hexagon in hexagon_amenities:
        hexagon_amenities[hexagon][amenity] += 1
    else:
        hexagon_amenities[hexagon] = {}
        for amenity in features_gdf['name'].unique():
            hexagon_amenities[hexagon][amenity] = 0
        hexagon_amenities[hexagon][amenity] += 1

# convert to a dataframe
# amenity_name, amenity_name2, amenity_name3, ...
# count, count2, count3, ...
hexagon_amenities_df = pd.DataFrame(hexagon_amenities).T

# count ohca in each hexagon
hexagon_ohca = {}
for ohca in ohca_gdf.iterrows():
    hexagon = h3.geo_to_h3(ohca[1]['geometry'].y, ohca[1]['geometry'].x, 9)
    if hexagon in hexagon_ohca:
        hexagon_ohca[hexagon] += 1
    else:
        hexagon_ohca[hexagon] = 1

# add ohca count to the hexagon_amenities_df
hexagon_amenities_df['ohca_count'] = [hexagon_ohca[h] if h in hexagon_ohca else 0 for h in hexagon_amenities_df.index]
display(hexagon_amenities_df.head())

,place_of_worship,grave_yard,school,post_office,childcare,university,courthouse,fire_station,library,police,...,tax_advisor,consulting,financial_advisor,camping,import_export_company,administrative,environment,stop_position,platform,ohca_count
892af0c9adbffff,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
892af052a6bffff,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
892af051c37ffff,1,2,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,1,1
892af0519b3ffff,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
892af0501b3ffff,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [50]:

import numpy as np

def add_neighbour_counts(hexagon_amenities_df: pd.DataFrame, feature_gdf: gpd.GeoDataFrame) -> pd.DataFrame:
    # Get unique features
    unique_features = feature_gdf['name'].unique()

    # Create new columns for each unique feature
    for amenity in unique_features:
        new_columns = pd.DataFrame({f"{amenity}_neighbour_count": 0}, index=hexagon_amenities_df.index)
        hexagon_amenities_df = pd.concat([hexagon_amenities_df, new_columns], axis=1)

    # Iterate through hexagon_amenities_df index
    for h in hexagon_amenities_df.index:
        neighbours = np.array(list(h3.k_ring(h, 1)))  # Convert set to array
        common_indices = np.intersect1d(neighbours, hexagon_amenities_df.index)
        
        # Increment the counts using vectorized operations
        hexagon_amenities_df.loc[h, unique_features + '_neighbour_count'] += hexagon_amenities_df.loc[common_indices, unique_features].values.sum(axis=0)

    # Ensure integer type for the new columns
    hexagon_amenities_df[unique_features + '_neighbour_count'] = hexagon_amenities_df[unique_features + '_neighbour_count'].astype(int)
    return hexagon_amenities_df

hexagon_amenities_df = add_neighbour_counts(hexagon_amenities_df, features_gdf)
display(hexagon_amenities_df.head())

,place_of_worship,grave_yard,school,post_office,childcare,university,courthouse,fire_station,library,police,...,healthcare_neighbour_count,tax_advisor_neighbour_count,consulting_neighbour_count,financial_advisor_neighbour_count,camping_neighbour_count,import_export_company_neighbour_count,administrative_neighbour_count,environment_neighbour_count,stop_position_neighbour_count,platform_neighbour_count
892af0c9adbffff,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
892af052a6bffff,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
892af051c37ffff,1,2,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,3
892af0519b3ffff,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
892af0501b3ffff,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [51]:

# get point data for warsaw
area_name = 'Warszawa'
warsaw_amenities = get_point_data(node_names[0], area_name, api)
warsaw_buildings = get_point_data(node_names[1], area_name, api)
warsaw_highways = get_point_data(node_names[2], area_name, api)
warsaw_public_transport = get_point_data(node_names[3], area_name, api)
warsaw_government = get_point_data(node_names[4], area_name, api)
warsaw_leisure = get_point_data(node_names[5], area_name, api)
warsaw_office = get_point_data(node_names[6], area_name, api)

warsaw_amenities.extend(warsaw_buildings)
warsaw_amenities.extend(warsaw_highways)
warsaw_amenities.extend(warsaw_public_transport)
warsaw_amenities.extend(warsaw_government)
warsaw_amenities.extend(warsaw_leisure)
warsaw_amenities.extend(warsaw_office)

# create gdf from amenities
# update amenities with changed building types
# update amenities with raw category types
warsaw_features_gdf = gpd.GeoDataFrame(data=[{'name': amen['type']} for amen in warsaw_amenities], geometry=[amen['geometry'] for amen in warsaw_amenities], crs='EPSG:4326')
# print number of unique names
display(warsaw_features_gdf.head())

,name,geometry
0,restaurant,POINT (20.94595 52.17691)
1,parking_entrance,POINT (21.01518 52.21904)
2,ferry_terminal,POINT (20.92711 52.33046)
3,shelter,POINT (21.04169 52.12263)
4,shelter,POINT (21.04179 52.11624)


In [52]:

# create a dict hexagon_amenities = {hexagon_id: {amenity_name: count}}
warsaw_hexagon_amenities = {}
for amen_dict in warsaw_features_gdf.iterrows():
    amenity = amen_dict[1]['name']
    hexagon = h3.geo_to_h3(amen_dict[1]['geometry'].y, amen_dict[1]['geometry'].x, 9)
    if hexagon in warsaw_hexagon_amenities:
        warsaw_hexagon_amenities[hexagon][amenity] += 1
    else:
        warsaw_hexagon_amenities[hexagon] = {}
        for amenity in warsaw_features_gdf['name'].unique():
            warsaw_hexagon_amenities[hexagon][amenity] = 0
        warsaw_hexagon_amenities[hexagon][amenity] += 1
    
# convert to a dataframe
# amenity_name, amenity_name2, amenity_name3, ...
# count, count2, count3, ...
warsaw_hexagon_amenities_df = pd.DataFrame(warsaw_hexagon_amenities).T

# add neighbour counts
warsaw_hexagon_amenities_df = add_neighbour_counts(warsaw_hexagon_amenities_df, warsaw_features_gdf)
display(warsaw_hexagon_amenities_df.head())
# save to csv
warsaw_hexagon_amenities_df.to_csv('warsaw_hexagon_amenities.csv')

,restaurant,parking_entrance,ferry_terminal,shelter,place_of_worship,fuel,fountain,parking,drinking_water,fast_food,...,translation_neighbour_count,administration_neighbour_count,graphic_design_neighbour_count,private_investigator_neighbour_count,tool_wholesaler_neighbour_count,international_organisation_neighbour_count,certified_translator_neighbour_count,interior_design_neighbour_count,stone_design_neighbour_count,geologist_neighbour_count
891f5224587ffff,5,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,6
891f53c9327ffff,22,3,0,0,2,0,0,0,0,8,...,0,0,2,0,0,2,0,0,0,7
891f53d9a13ffff,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
891f5224ba3ffff,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
891f5224b23ffff,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [54]:
from xgboost import XGBRegressor

# remove columns that are not in the training data
for column in warsaw_hexagon_amenities_df.columns:
    if column not in hexagon_amenities_df.columns:
        del warsaw_hexagon_amenities_df[column]

# remove columns that are not in the data to predict
for column in hexagon_amenities_df.columns:
    if column not in warsaw_hexagon_amenities_df.columns and column != 'ohca_count':
        del hexagon_amenities_df[column]
    
# train model
model = XGBRegressor(max_depth=10)
X = hexagon_amenities_df.drop(columns=['ohca_count'])
# sort columns in both dataframes
X = X.reindex(sorted(X.columns), axis=1)
warsaw_hexagon_amenities_df = warsaw_hexagon_amenities_df.reindex(sorted(warsaw_hexagon_amenities_df.columns), axis=1)
# try to add warsaw data to the training data
y = hexagon_amenities_df['ohca_count']
model.fit(X, y)

# predict
X = warsaw_hexagon_amenities_df
y_pred = model.predict(X)
y_pred = np.maximum(y_pred, 0)

# add predictions to the warsaw_hexagon_amenities_df
warsaw_hexagon_amenities_df['predicted_ohca_count'] = y_pred
# create a map, color hexagons by the predicted number of ohca
import folium

m = folium.Map(location=[52.2297, 21.0122], zoom_start=11)

# add hexagons with opacity based on the number of ohca
for hexagon in warsaw_hexagon_amenities_df.index:
    # get ohca from the hexagon_amenities_df
    ohca = warsaw_hexagon_amenities_df.loc[hexagon, 'predicted_ohca_count']
    opacity = ohca / 10
    color = 'red'
    locs = [(pos[0], pos[1]) for pos in h3.h3_to_geo_boundary(hexagon)]
    # create a polygon from the hexagon
    folium.Polygon(locations=locs, color=color, fill_color=color, fill_opacity=opacity).add_to(m)

m